**Dodatek D – Różniczkowanie automatyczne**

_Notatnik ten zawiera przykładowe implementacje różnych technik różniczkowania automatycznego, pozwalające zrozumieć mechanizm ich działania._

# Konfiguracja

# Wprowadzenie

Załóżmy, że chcemy obliczyć gradienty funkcji $f(x,y)=x^2y + y + 2$ w odniesieniu do parametrów x i y:

In [2]:
def f(x,y):
    return x*x*y + y + 2

Możemy dokonać tego analitycznie:

$\dfrac{\partial f}{\partial x} = 2xy$

$\dfrac{\partial f}{\partial y} = x^2 + 1$

In [3]:
def df(x,y):
    return 2*x*y, x*x + 1

Zatem, na przykład $\dfrac{\partial f}{\partial x}(3,4) = 24$ i $\dfrac{\partial f}{\partial y}(3,4) = 10$.

In [4]:
df(3, 4)

(24, 10)

Doskonale! Możemy również określić równania dla pochodnych drugiego rzędu (hesjanów):

$\dfrac{\partial^2 f}{\partial x \partial x} = \dfrac{\partial (2xy)}{\partial x} = 2y$

$\dfrac{\partial^2 f}{\partial x \partial y} = \dfrac{\partial (2xy)}{\partial y} = 2x$

$\dfrac{\partial^2 f}{\partial y \partial x} = \dfrac{\partial (x^2 + 1)}{\partial x} = 2x$

$\dfrac{\partial^2 f}{\partial y \partial y} = \dfrac{\partial (x^2 + 1)}{\partial y} = 0$

Hesjany te w punkcie x=3 i y=4, wynoszą, odpowiednio, 8, 6, 6, 0. Użyjmy powyższych równań do ich obliczenia:

In [5]:
def d2f(x, y):
    return [2*y, 2*x], [2*x, 0]

In [6]:
d2f(3, 4)

([8, 6], [6, 0])

Znakomicie, ale rozwiązanie to wymaga nieco zaangażowania matematycznego. W tym przypadku nie było zbyt trudno, ale w przypadku głębokich sieci neuronowych obliczenie pochodnych w ten sposób okazuje się niemal niemożliwe. Zobaczmy więc, jak możemy zautomatyzować ten proces!

# Różniczkowanie numeryczne

Obliczamy tu wartość przybliżoną gradientów za pomocą równania: $\dfrac{\partial f}{\partial x} = \displaystyle{\lim_{\epsilon \to 0}}\dfrac{f(x+\epsilon, y) - f(x, y)}{\epsilon}$ (i występuje podobna definicja dla $\dfrac{\partial f}{\partial y}$).

In [7]:
def gradients(func, vars_list, eps=0.0001):
    partial_derivatives = []
    base_func_eval = func(*vars_list)
    for idx in range(len(vars_list)):
        tweaked_vars = vars_list[:]
        tweaked_vars[idx] += eps
        tweaked_func_eval = func(*tweaked_vars)
        derivative = (tweaked_func_eval - base_func_eval) / eps
        partial_derivatives.append(derivative)
    return partial_derivatives

In [8]:
def df(x, y):
    return gradients(f, [x, y])

In [9]:
df(3, 4)

[24.000400000048216, 10.000000000047748]

Rozwiązanie to działa dobrze!

Dobra wiadomość jest tak, że całkiem łatwo można obliczać w ten sposób hesjany. Stwórzmy najpierw funkcję, która oblicza pochodne pierwszego rzędu (jakobiany): 

In [10]:
def dfdx(x, y):
    return gradients(f, [x,y])[0]

def dfdy(x, y):
    return gradients(f, [x,y])[1]

dfdx(3., 4.), dfdy(3., 4.)

(24.000400000048216, 10.000000000047748)

Wystarczy teraz użyć funkcji `gradients()` na tych funkcjach:

In [11]:
def d2f(x, y):
    return [gradients(dfdx, [3., 4.]), gradients(dfdy, [3., 4.])]

In [12]:
d2f(3, 4)

[[7.999999951380232, 6.000099261882497],
 [6.000099261882497, -1.4210854715202004e-06]]

Zatem wszystko działa, jak należy, ale wynik jest przybliżeniem, a obliczenie gradientów funkcji w odniesieniu do $n$ zmiennych wymaga wywołania tej funkcji $n$ razy. W głębokich sieciach neuronowych często występują tysiące parametrów, które są modyfikowanie w fazie gradientu prostego (gdy jest wymagane obliczenie gradientów funkcji straty w odniesieniu do każdego z tych parametrów), więc to rozwiązanie jest zbyt powolne.

## Implementacja przykładowego grafu obliczeniowego

Pomińmy rozwiązanie numeryczne i zaimplementujmy pewne technika symbolicznego różniczkowania automatycznego. W tym celu musimy zdefiniować klasy reprezentujące stałe, zmienne i operacje.

In [13]:
class Const(object):
    def __init__(self, value):
        self.value = value
    def evaluate(self):
        return self.value
    def __str__(self):
        return str(self.value)

class Var(object):
    def __init__(self, name, init_value=0):
        self.value = init_value
        self.name = name
    def evaluate(self):
        return self.value
    def __str__(self):
        return self.name

class BinaryOperator(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

class Add(BinaryOperator):
    def evaluate(self):
        return self.a.evaluate() + self.b.evaluate()
    def __str__(self):
        return "{} + {}".format(self.a, self.b)

class Mul(BinaryOperator):
    def evaluate(self):
        return self.a.evaluate() * self.b.evaluate()
    def __str__(self):
        return "({}) * ({})".format(self.a, self.b)

Dobrze, możemy teraz stworzyć graf obliczeniowy, reprezentujący funkcję $f$:

In [14]:
x = Var("x")
y = Var("y")
f = Add(Mul(Mul(x, x), y), Add(y, Const(2))) # f(x,y) = x²y + y + 2

Możemy uruchomić ten graf, aby obliczał funkcję $f$ w dowolnym punkcie, na przykład $f(3, 4)$.

In [15]:
x.value = 3
y.value = 4
f.evaluate()

42

Świetnie, uzyskaliśmy ostateczną odpowiedź. 

## Obliczanie gradientów

Poniższe metody różniczkowania automatycznego bazują na *regule łańcuchowej*.

Załóżmy, że mamy dwie funkcje, $u$ i $v$, i przekazujemy je sekwencyjnie do jakiegoś wejścia $x$, po czym otrzymujemy wynik $z$. Otrzymujemy więc $z = v(u(x))$, co możemy również zapisać jako $z = v(s)$ i $s = u(x)$. Możemy teraz zastosować regułę łańcuchową, aby uzyskać pochodną cząstkową wyjścia $z$ w odniesieniu do wejścia $x$:

$ \dfrac{\partial z}{\partial x} = \dfrac{\partial s}{\partial x} \cdot \dfrac{\partial z}{\partial s}$

Jeżeli $z$ jest wynikiem sekwencji funkcji, w której występują wyniki pośrednie $s_1, s_2, ..., s_n$, to reguła łańcuchowa ciągle będzie miała rację bytu: 

$ \dfrac{\partial z}{\partial x} = \dfrac{\partial s_1}{\partial x} \cdot \dfrac{\partial s_2}{\partial s_1} \cdot \dfrac{\partial s_3}{\partial s_2} \cdot \dots \cdot \dfrac{\partial s_{n-1}}{\partial s_{n-2}} \cdot \dfrac{\partial s_n}{\partial s_{n-1}} \cdot \dfrac{\partial z}{\partial s_n}$

W klasycznym różniczkowaniu automatycznym algorytm oblicza te wyrażenia "w przód" (tzn. w tej samej kolejności, co obliczenia wymagane do uzyskania wyniku $z$), czyli od lewej do prawej: najpierw $\dfrac{\partial s_1}{\partial x}$, następnie $\dfrac{\partial s_2}{\partial s_1}$ itd. W odwrotnym różniczkowaniu automatycznym algorytm oblicza te wyrażenia "wstecz", od prawej do lewej: najpierw $\dfrac{\partial z}{\partial s_n}$, następnie $\dfrac{\partial s_n}{\partial s_{n-1}}$ itd.

Załóżmy, na przykład,że chcesz obliczyć pochodną funkcji $z(x)=\sin(x^2)$ w punkcie x=3, za pomocą klasycznego różniczkowania automatycznego. Algorytm obliczyłby najpierw pochodną cząstkową $\dfrac{\partial s_1}{\partial x}=\dfrac{\partial x^2}{\partial x}=2x=6$. Następnie przeszedłby do obliczenia $\dfrac{\partial z}{\partial x}=\dfrac{\partial s_1}{\partial x}\cdot\dfrac{\partial z}{\partial s_1}= 6 \cdot \dfrac{\partial \sin(s_1)}{\partial s_1}=6 \cdot \cos(s_1)=6 \cdot \cos(3^2)\approx-5,46$.

Zweryfikujmy ten wynik za pomocą zdefiniowanej wcześniej funkcji `gradients()`:

In [16]:
from math import sin

def z(x):
    return sin(x**2)

gradients(z, [3])

[-5.46761419430053]

Wygląda to nieżle. Przeprowadźmy tę samą operację za pomocą odwrotnego różniczkowania automatycznego. Tym razem algorytm rozpocznie z prawej strony i obliczy $\dfrac{\partial z}{\partial s_1} = \dfrac{\partial \sin(s_1)}{\partial s_1}=\cos(s_1)=\cos(3^2)\approx -0,91$. Następnie zajmie się obliczeniam $\dfrac{\partial z}{\partial x}=\dfrac{\partial s_1}{\partial x}\cdot\dfrac{\partial z}{\partial s_1} \approx \dfrac{\partial s_1}{\partial x} \cdot -0,91 = \dfrac{\partial x^2}{\partial x} \cdot -0,91=2x \cdot -0,91 = 6\cdot-0,91=-5,46$.

Obydwie metody dają, oczywiście, ten sam wynik (nie licząc błędów zaokrąglenia) i w obecności jednego wejścia oraz wyjścia przeprowadzają identyczną liczbę obliczeń. Jednak w przypadku występowania kilku wejść lub wyjść możemy uzyskiwać bardzo odmienną wydajność. Rzeczywiście, jeżeli występuje wiele wejść, wyrażenia znajdujące się skrajnie po prawej stronie będą musiały obliczać pochodne cząstkowe w odniesieniu do każdego wejścia, dlatego warto je obliczyć jako pierwsze. Oznacza to potrzebę użycia odwrotnego różniczkowania automatycznego. W ten sposób wyrażenia znajdujące się skrajnie po prawej stronie zostaną obliczone tylko i wykorzystanie do obliczenia wszystkich pochodnych cząstkowych. Z drugiej strony, w przypadku występowania wielu wyjść, zazwyczaj lepiej korzystać z klasycznego różniczkowania automatycznego, ponieważ wystarczy obliczyć raz wyrażenia znajdujące się skrajnie po lewej stronie i użyte do obliczenia wszystkich pochodnych cząstkowych dla różnych wyjść. W uczeniu głębokim występują zazwyczaj tysiące parametrów modelu, co oznacza występowania wielu wejść i tylko kilka wyjść. W rzeczywistości, w trakcie uczenia występuje tylko jeden wynik: wartość funkcji straty. Z tego właśnie powodu odwrotne różniczkowanie automatyczne jest używane w module TensorFlow i wszystkich najważniejszych bibliotekach uczenia głębokiego.  

W odwrotnym różniczkowaniu automatycznym należy wziąć pod uwagę jeszcze jedną kwestię: wartość $s_i$ jest zazwyczaj wymagana podczas obliczania $\dfrac{\partial s_{i+1}}{\partial s_i}$, natomiast w celu obliczenia $s_i$ należy najpierw obliczyć $s_{i-1}$, a to wymaga obliczenia $s_{i-2}$ itd. Zasadniczo więc pierwszy przebieg w przód przez sieć jest wymagany do obliczenia $s_1$, $s_2$, $s_3$, $\dots$, $s_{n-1}$ i $s_n$, po czym algorytm może obliczyć pochodne cząstkowe w przeciwnym kierunku. Przechowywanie wszystkich wartości pośrednich $s_i$ w pamięci operacyjnej stanowi czasami problem, zwłaszcza w przypadku przetwarzania obrazów oraz podczas korzystania z karty graficznej, w której pamięć RAM jest nieraz dość ograniczona: aby załagodzić ten problem można zmniejszyć liczbę warstw w sieci neuronowej albo tak skonfigurować moduł TensorFlow, aby wartości te były przenoszone z pamięci operacyjnej karty graficznej do głównej pamięci RAM. Innym rozwiązaniem jest przechowywanie w pamięci podręcznej jedynie co drugiej wartości pośredniej, $s_1$, $s_3$, $s_5$, $\dots$, $s_{n-4}$, $s_{n-2}$ i $s_n$. Oznacza to, że podczas obliczania pochodnych cząstkowych i braku wartości pośredniej $s_i$, należy ją ponownie obliczyć na podstawie poprzedniej wartości pośredniej $s_{i-1}$. Poświęcamy tu moc obliczeniową na rzecz pamięci operacyjnej (więcej informacji na ten temat znajdziesz w [tym artykule](https://pdfs.semanticscholar.org/f61e/9fd5a4878e1493f7a6b03774a61c17b7e9a4.pdf)).

### Klasyczne różnicznkowanie automatyczne

In [17]:
Const.gradient = lambda self, var: Const(0)
Var.gradient = lambda self, var: Const(1) if self is var else Const(0)
Add.gradient = lambda self, var: Add(self.a.gradient(var), self.b.gradient(var))
Mul.gradient = lambda self, var: Add(Mul(self.a, self.b.gradient(var)), Mul(self.a.gradient(var), self.b))

x = Var(name="x", init_value=3.)
y = Var(name="y", init_value=4.)
f = Add(Mul(Mul(x, x), y), Add(y, Const(2))) # f(x,y) = x²y + y + 2

dfdx = f.gradient(x)  # 2xy
dfdy = f.gradient(y)  # x² + 1

In [18]:
dfdx.evaluate(), dfdy.evaluate()

(24.0, 10.0)

Wynik metody `gradient()` jest całkowicie symboliczny, dlatego nie musimy ograniczać się wyłącznie do pochodnych pierwszego rzędu i możemy obliczać pochodne rzędu drugiego itd.:

In [19]:
d2fdxdx = dfdx.gradient(x) # 2y
d2fdxdy = dfdx.gradient(y) # 2x
d2fdydx = dfdy.gradient(x) # 2x
d2fdydy = dfdy.gradient(y) # 0

In [20]:
[[d2fdxdx.evaluate(), d2fdxdy.evaluate()],
 [d2fdydx.evaluate(), d2fdydy.evaluate()]]

[[8.0, 6.0], [6.0, 0.0]]

Zwróć uwagę, że wynik jest teraz dokładny i nie stanowi przybliżenia (oczywiście jedynie ograniczenie stanowi precyzja zmiennoprzecinkowa komputera).

### Klasyczne różniczkowanie automatyczne za pomocą liczb dualnych 

Ciekawym sposobem stosowania klasycznego różniczkowania automatycznego jest korzystanie z [liczb dualnych](https://pl.wikipedia.org/wiki/Liczby_dualne). Mówiąc krótko, liczba dualna $z$ przyjmuje postać $z = a + b\epsilon$, gdzie $a$ i $b$ są liczbami rzeczywistymi, natomiast $\epsilon$ jest nieskończenie małą wartością, dodatnią, ale mniejszą od jakiejkolwiek liczby rzeczywistej, tak że $\epsilon^2=0$.
Można udowodnić, że $f(x + \epsilon) = f(x) + \dfrac{\partial f}{\partial x}\epsilon$, zatem wystarczy obliczyć $f(x + \epsilon)$, aby otrzymać zarówno wartość $f(x)$, jak i pochodną cząstkową $f$ w odniesieniu do $x$. 

Liczby dualne mają własne reguły arytmetyczne, które są zasadniczo całkiem intuicyjne. Na przykład: 

**Dodawanie**

$(a_1 + b_1\epsilon) + (a_2 + b_2\epsilon) = (a_1 + a_2) + (b_1 + b_2)\epsilon$

**Odejmowanie**

$(a_1 + b_1\epsilon) - (a_2 + b_2\epsilon) = (a_1 - a_2) + (b_1 - b_2)\epsilon$

**Mnożenie**

$(a_1 + b_1\epsilon) \times (a_2 + b_2\epsilon) = (a_1 a_2) + (a_1 b_2 + a_2 b_1)\epsilon + b_1 b_2\epsilon^2 = (a_1 a_2) + (a_1b_2 + a_2b_1)\epsilon$

**Dzielenie**

$\dfrac{a_1 + b_1\epsilon}{a_2 + b_2\epsilon} = \dfrac{a_1 + b_1\epsilon}{a_2 + b_2\epsilon} \cdot \dfrac{a_2 - b_2\epsilon}{a_2 - b_2\epsilon} = \dfrac{a_1 a_2 + (b_1 a_2 - a_1 b_2)\epsilon - b_1 b_2\epsilon^2}{{a_2}^2 + (a_2 b_2 - a_2 b_2)\epsilon - {b_2}^2\epsilon} = \dfrac{a_1}{a_2} + \dfrac{a_1 b_2 - b_1 a_2}{{a_2}^2}\epsilon$

**Potęgowanie**

$(a + b\epsilon)^n = a^n + (n a^{n-1}b)\epsilon$

itd.

Stwórzmy klasę reprezentującą liczby dualne i zaimplementujmy kilka operacji (dodawanie i mnożenie). Jeśli chcesz, możesz spróbować dołączyć również własne operacje.

In [21]:
class DualNumber(object):
    def __init__(self, value=0.0, eps=0.0):
        self.value = value
        self.eps = eps
    def __add__(self, b):
        return DualNumber(self.value + self.to_dual(b).value,
                          self.eps + self.to_dual(b).eps)
    def __radd__(self, a):
        return self.to_dual(a).__add__(self)
    def __mul__(self, b):
        return DualNumber(self.value * self.to_dual(b).value,
                          self.eps * self.to_dual(b).value + self.value * self.to_dual(b).eps)
    def __rmul__(self, a):
        return self.to_dual(a).__mul__(self)
    def __str__(self):
        if self.eps:
            return "{:.1f} + {:.1f}ε".format(self.value, self.eps)
        else:
            return "{:.1f}".format(self.value)
    def __repr__(self):
        return str(self)
    @classmethod
    def to_dual(cls, n):
        if hasattr(n, "value"):
            return n
        else:
            return cls(n)

$3 + (3 + 4 \epsilon) = 6 + 4\epsilon$

In [22]:
3 + DualNumber(3, 4)

6.0 + 4.0ε

$(3 + 4ε)\times(5 + 7ε)$ = $3 \times 5 + 3 \times 7ε + 4ε \times 5 + 4ε \times 7ε$ = $15 + 21ε + 20ε + 28ε^2$ = $15 + 41ε + 28 \times 0$ = $15 + 41ε$

In [23]:
DualNumber(3, 4) * DualNumber(5, 7)

15.0 + 41.0ε

Sprawdźmy, czy liczby dual będą współdziałać z naszym przykładowym środowiskiem:

In [24]:
x.value = DualNumber(3.0)
y.value = DualNumber(4.0)

f.evaluate()

42.0

Tak, zdecydowanie współdziałają. Wykorzystajmy je teraz do obliczenia pochodnych cząstkowych $f$ w odniesieniu do $x$ i $y$ w punktach x=3 i y=4:

In [25]:
x.value = DualNumber(3.0, 1.0)  # 3 + ε
y.value = DualNumber(4.0)       # 4

dfdx = f.evaluate().eps

x.value = DualNumber(3.0)       # 3
y.value = DualNumber(4.0, 1.0)  # 4 + ε

dfdy = f.evaluate().eps

In [26]:
dfdx

24.0

In [27]:
dfdy

10.0

Cudownie! Implementacja ta jest jednak ograniczona do pochodnych pierwszego rzędu. 
Przejdźmy do wariantu odwrotnego.

### Odwrotne różniczkowanie automatyczne

Zmodyfikujmy nasze środowisko przykładowe i dodajmy odwrotne różniczkowanie automatyczne:

In [28]:
class Const(object):
    def __init__(self, value):
        self.value = value
    def evaluate(self):
        return self.value
    def backpropagate(self, gradient):
        pass
    def __str__(self):
        return str(self.value)

class Var(object):
    def __init__(self, name, init_value=0):
        self.value = init_value
        self.name = name
        self.gradient = 0
    def evaluate(self):
        return self.value
    def backpropagate(self, gradient):
        self.gradient += gradient
    def __str__(self):
        return self.name

class BinaryOperator(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

class Add(BinaryOperator):
    def evaluate(self):
        self.value = self.a.evaluate() + self.b.evaluate()
        return self.value
    def backpropagate(self, gradient):
        self.a.backpropagate(gradient)
        self.b.backpropagate(gradient)
    def __str__(self):
        return "{} + {}".format(self.a, self.b)

class Mul(BinaryOperator):
    def evaluate(self):
        self.value = self.a.evaluate() * self.b.evaluate()
        return self.value
    def backpropagate(self, gradient):
        self.a.backpropagate(gradient * self.b.value)
        self.b.backpropagate(gradient * self.a.value)
    def __str__(self):
        return "({}) * ({})".format(self.a, self.b)

In [29]:
x = Var("x", init_value=3)
y = Var("y", init_value=4)
f = Add(Mul(Mul(x, x), y), Add(y, Const(2))) # f(x,y) = x²y + y + 2

result = f.evaluate()
f.backpropagate(1.0)

In [30]:
print(f)

((x) * (x)) * (y) + y + 2


In [31]:
result

42

In [32]:
x.gradient

24.0

In [33]:
y.gradient

10.0

Również w tej implementacji wyniki są jedynie wartościami liczbowymi, a nie wyrażeniami symbolicznymi, dlatego jesteśmy ograniczeni do pochodnych pierwszego rzędu. Moglibyśmy jednak sprawić, żeby metody `backpropagate()` zwracały wyrażenia symboliczne zamiast wartości (np. żeby zwracały `Add(2,3)` zamiast 5). W ten sposób moglibyśmy obliczać gradienty drugiego rzędu (i wyższe). To samo robi implementacja w module TensorFlow, jak również wszystkie główne biblioteki zawierające operację różniczkowania automatycznego.

### Odwrotne różniczkowanie automatyczne w module TensorFlow

In [34]:
import tensorflow as tf

In [35]:
tf.reset_default_graph()

x = tf.Variable(3., name="x")
y = tf.Variable(4., name="y")
f = x*x*y + y + 2

jacobians = tf.gradients(f, [x, y])

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    f_val, jacobians_val = sess.run([f, jacobians])

f_val, jacobians_val

(42.0, [24.0, 10.0])

Wszystkie wyrażenia są tu symboliczne, dlatego możemy obliczyć pochodne drugiego rzędu i wyższe. Jeżeli jednak obliczymy pochodną tensora w odniesieniu do zmiennej, od której nie jest zależny, to nie otrzymamy w rezultacie 0.0, lecz funkcja `gradients()` zwróci wartość None, której nie można wykorzystać w metodzie `sess.run()`. Uważaj więc na wartości `None`. Zastępujemy je tu tensorami zerowymi.

In [36]:
hessians_x = tf.gradients(jacobians[0], [x, y])
hessians_y = tf.gradients(jacobians[1], [x, y])

def replace_none_with_zero(tensors):
    return [tensor if tensor is not None else tf.constant(0.)
            for tensor in tensors]

hessians_x = replace_none_with_zero(hessians_x)
hessians_y = replace_none_with_zero(hessians_y)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    hessians_x_val, hessians_y_val = sess.run([hessians_x, hessians_y])

hessians_x_val, hessians_y_val

([8.0, 6.0], [6.0, 0.0])

I to by było na tyle! Mam nadzieję, że spodobał Ci się ten notatnik.